In [52]:
import pandas as pd
from pymongo import MongoClient
from flask import Flask, jsonify, request, send_from_directory
from flask_cors import CORS
import json
import webbrowser
import os


In [53]:
app = Flask(__name__)
CORS(app)

In [54]:
client = MongoClient(port=27017)
db = client['yelp_database']
collection = db['businesses']

In [55]:

# Read JSON file into a pandas DataFrame. Use lines=true because of the json format being a serie of jsons, one on each line.
df = pd.read_json('yelp_academic_dataset_business.json', lines=True)

In [56]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [57]:
df['categories'] = df['categories'].astype(str)

In [58]:
# Filter dataframe down to only businesses with restaurant in their categories
df_restaurants = df[df['categories'].str.contains('Restaurants', case=False, na=False)]

In [59]:
len(df_restaurants)


52268

In [60]:
## list unique restaurant categories ##

# Step 1: Split the categories and create a flat list
all_categories = df_restaurants['categories'].str.split(', ').explode()

# Step 2: Get unique values and sort them
unique_categories = sorted(all_categories.unique())

# Step 3: Print the list of unique categories
print("Unique categories:")
for category in unique_categories:
    print(category)

# If you want to know the total number of unique categories:
print(f"\nTotal number of unique categories: {len(unique_categories)}")

Unique categories:
Acai Bowls
Accessories
Accountants
Active Life
Acupuncture
Adult
Adult Education
Adult Entertainment
Advertising
Afghan
African
Air Duct Cleaning
Airlines
Airport Lounges
Airport Shuttles
Airport Terminals
Airports
Amateur Sports Teams
American (New)
American (Traditional)
Amusement Parks
Animal Shelters
Antiques
Apartments
Appliances
Appliances & Repair
Aquariums
Arabic
Arcades
Argentine
Armenian
Art Classes
Art Galleries
Art Museums
Art Tours
Arts & Crafts
Arts & Entertainment
Asian Fusion
Auction Houses
Audio/Visual Equipment Rental
Australian
Austrian
Auto Customization
Auto Detailing
Auto Glass Services
Auto Loan Providers
Auto Parts & Supplies
Auto Repair
Automotive
Axe Throwing
Bagels
Bakeries
Bangladeshi
Banks & Credit Unions
Bar Crawl
Barbeque
Barbers
Bars
Bartenders
Basque
Battery Stores
Batting Cages
Beach Bars
Beaches
Beauty & Spas
Bed & Breakfast
Beer
Beer Bar
Beer Gardens
Beer Hall
Beer Tours
Belgian
Beverage Store
Bike Rentals
Bike Repair/Maintenance
B

In [61]:

# Step 1: Split the categories and create a flat list
all_categories = df_restaurants['categories'].str.split(', ').explode()

# Step 2: Count the occurrences of each category
category_counts = all_categories.value_counts().sort_values()

# Step 3: Print the results
print("Unique categories and their counts (from smallest to largest):")
for category, count in category_counts.items():
    print(f"{category}: {count}")

# Print total number of unique categories
print(f"\nTotal number of unique categories: {len(category_counts)}")

Unique categories and their counts (from smallest to largest):
Gemstones & Minerals: 1
Divorce & Family Law: 1
Auto Loan Providers: 1
Airlines: 1
Health Retreats: 1
Makeup Artists: 1
Hotel bar: 1
Special Education: 1
Water Stores: 1
Wedding Chapels: 1
Car Stereo Installation: 1
Trailer Repair: 1
Haunted Houses: 1
RV Repair: 1
Smog Check Stations: 1
Conveyor Belt Sushi: 1
Trivia Hosts: 1
Personal Assistants: 1
Legal Services: 1
General Litigation: 1
Employment Law: 1
Bridal: 1
Tonkatsu: 1
Ethical Grocery: 1
Contract Law: 1
Business Law: 1
Personal Injury Law: 1
Trains: 1
Fire Departments: 1
Air Duct Cleaning: 1
Radio Stations: 1
Real Estate Law: 1
Commercial Truck Repair: 1
Nail Technicians: 1
Security Systems: 1
Cucina campana: 1
Flight Instruction: 1
Transmission Repair: 1
Hindu Temples: 1
Countertop Installation: 1
Reiki: 1
Roadside Assistance: 1
Windshield Installation & Repair: 1
Internet Service Providers: 1
Irrigation: 1
Diagnostic Imaging: 1
Skate Shops: 1
Oral Surgeons: 1
Lives

In [62]:
# search categories
search_term = "station"
zoo_businesses = df_restaurants[df_restaurants['categories'].str.contains(search_term, case=False, na=False)]

# Display the results
print(f"Number of businesses with {search_term} in their category: {len(zoo_businesses)}")

# Display the first few rows of the results
print(zoo_businesses[['name', 'city', 'state', 'categories']].head())

# If you want to see all unique category combinations that include the searched category
zoo_categories = zoo_businesses['categories'].unique()
print(f"\nUnique category combinations including {search_term}:")
for categories in zoo_categories:
    print(categories)

Number of businesses with station in their category: 263
          name          city state  \
82        Wawa  Philadelphia    PA   
835       Wawa    Norristown    PA   
940       Wawa  Philadelphia    PA   
1440      Wawa      Narberth    PA   
1975  QuikTrip      St Louis    MO   

                                             categories  
82    Restaurants, Automotive, Delis, Gas Stations, ...  
835   Food, Coffee & Tea, Gas Stations, Restaurants,...  
940   Delis, Gas Stations, Food, Convenience Stores,...  
1440  Gas Stations, Food, Automotive, Restaurants, S...  
1975  Automotive, Gas Stations, Food, Convenience St...  

Unique category combinations including station:
Restaurants, Automotive, Delis, Gas Stations, Food, Coffee & Tea, Sandwiches, Convenience Stores
Food, Coffee & Tea, Gas Stations, Restaurants, Automotive, Sandwiches, Convenience Stores
Delis, Gas Stations, Food, Convenience Stores, Restaurants, Automotive, Coffee & Tea, Sandwiches
Gas Stations, Food, Automotive, R

In [63]:
strict_restaurant_categories = [
    "Restaurants", "American (Traditional)", "American (New)", "Italian", 
    "Mexican", "Chinese", "Japanese", "Thai", "Vietnamese", "Indian", "Korean", 
    "Mediterranean", "Greek", "French", "Spanish", "German", "Irish", "British", 
    "Seafood", "Steakhouses", "Pizza", "Burgers", "Sushi Bars", 
    "Barbeque", "Asian Fusion", "Diners", "Breakfast & Brunch", 
    "Buffets", "Comfort Food", "Soul Food", "Southern", 
    "Tex-Mex", "Cajun/Creole", "Latin American", "Middle Eastern", "African", 
    "Caribbean", "Brazilian", "Peruvian", "Cuban", "Tapas Bars", "Gastropubs",
    "Izakaya", "Ramen", "Poke", "Hot Pot", "Dim Sum", "Fondue",
    "Fish & Chips", "Bistros", "Brasseries",
    "Donairs", "Kebab", "Falafel", "Tacos", "Cheesesteaks"
]


food_related_categories = [
    "Food", "Sandwiches", "Cafes", "Fast Food", "Pubs", 
    "Halal", "Kosher", "Vegan", "Vegetarian",
    "Gluten-Free", "Organic", "Farm-to-table", "Food Trucks", "Food Stands",
    "Creperies", "Delis", "Noodles", "Soup", "Salad", "Chicken Wings",
    "Chicken Shop", "Hot Dogs", "Waffles", "Pancakes", "Donuts", "Bagels",
    "Food", "Specialty Food", "Caterers", "Bakeries", "Desserts", "Juice Bars & Smoothies",
    "Ice Cream & Frozen Yogurt", "Gelato", "Coffee & Tea"
]

In [64]:
 # Select relevant columns
df_restaurants_clean = df_restaurants[['business_id', 'name', 'city', 'state', 'latitude', 'longitude', 'categories', 'stars']]
df_restaurants_clean.head()

,business_id,name,city,state,latitude,longitude,categories,stars
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",4.0
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",2.0
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,38.565165,-90.321087,"Pubs, Restaurants, Italian, Bars, American (Tr...",3.0
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,36.208102,-86.768170,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",1.5
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,Tampa Bay,FL,27.955269,-82.456320,"Vietnamese, Food, Restaurants, Food Trucks",4.0


In [65]:
# collection.delete_many({})

In [66]:
# Connect to MongoDB
if collection.count_documents({}) == 0:
    records = df_restaurants_clean.to_dict('records')
    collection.insert_many(records)
    print("Data loaded into MongoDB successfully.")
else:
    print("Data already exists in MongoDB.")

Data already exists in MongoDB.


In [67]:
# Setup flask api routes
@app.route('/categories', methods=['GET'])
def get_categories():
    return jsonify({
        'strict_restaurant_categories': strict_restaurant_categories
    })


In [68]:

@app.route('/query_restaurants/<category>', methods=['GET'])
def query_restaurants(category):
    query = {"categories": {'$regex': category, '$options': 'i'}}

    # Get results from MongoDB
    results = list(collection.find(query))

    # Convert ObjectId to string for JSON serialization
    for result in results:
        result['_id'] = str(result['_id'])

    return jsonify(results)


In [69]:
base_dir = os.path.dirname(os.path.abspath(__file__))
# Serve HTML file
@app.route('/')
def serve_html():
    return send_from_directory(base_dir, 'index.html')

# Serve JavaScript file
@app.route('/<path:path>')
def serve_file(path):
    return send_from_directory(base_dir, path)

In [73]:
# webbrowser.open('http://localhost:5000/')
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2024-10-22 02:54:46,399] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "c:\Users\AdamKarner\anaconda3\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\AdamKarner\anaconda3\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\AdamKarner\anaconda3\Lib\site-packages\flask_cors\extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "c:\Users\AdamKarner\anaconda3\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\AdamKarner\anaconda3\Lib\site-packages\flask